# Web Scraping for DataViz Exam Project - Football Transfers

In [3]:
# imports
import bs4 as bs
import requests
import pandas as pd
from tqdm import tqdm
import numpy as np
import pickle
import sys
import pyarrow as pa

In [4]:
def create_soup(url):
    headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                         "AppleWebKit/605.1.15 (KHTML, like Gecko) "
                         "Version/15.4 Safari/605.1.15"}
    page = requests.get(url, headers=headers)
    soup = bs.BeautifulSoup(page.content,'html.parser')
    return soup

In [3]:
url = 'https://www.transfermarkt.com/premier-league/transfers/wettbewerb/GB1/plus/?saison_id=2018&s_w=&leihe=1&intern=0&intern=1'
soup = create_soup(url)
main_league = 'Premier League'
main_season = '18/19'

In [121]:
# initializing the league_dict and club_info for later use
league_dict = {}
club_info = []

### Getting a list of tuples for all clubs

In [ ]:
# finding club id and club code for each club 
boxes = soup.find('div', class_='large-8 columns').find_all('div', class_='box')

for box in tqdm(boxes[3:]):
    club_name = box.find('img').get('alt')
    print(f'club_name: {club_name}')
    tables = box.find_all('div', class_='responsive-table')
    ins = tables[0].find('tbody').find_all('tr')
    outs = tables[1].find('tbody').find_all('tr')

    for row in ins:
        fee = row.find_all('td', class_='rechts')[1].string
        if isinstance(fee, str):
            fee_first = fee[0]
            if fee_first == "€":
                club_section = row.find('td', class_='no-border-links verein-flagge-transfer-cell').find('a')
                club_url = club_section.get('href')
                link_list = club_url.split('/')
                club_code = link_list[1]
                club_id = link_list[4]
                club_info.append((club_code, club_id))
    
    for row in outs:
        fee = row.find_all('td', class_='rechts')[1].string
        if isinstance(fee, str):
            fee_first = fee[0]
            if fee_first == "€":
                club_section = row.find('td', class_='no-border-links verein-flagge-transfer-cell').find('a')
                club_url = club_section.get('href')
                link_list = club_url.split('/')
                club_code = link_list[1]
                club_id = link_list[4]
                club_info.append((club_code, club_id))

In [53]:
# saving the club_info_set as a set in a pickle file
# with open('my_set.pkl', 'wb') as file:
#     pickle.dump(club_info_set, file)

# loading the pickle file containing the set of tuples
with open('club_tuples.pkl', 'rb') as file:
    loaded_set = pickle.load(file)

657


### Extracting league info for each club to create the league dictionary

In [57]:
# extracting league information for each team in the set of tuples
league_dict = {}
for club_info_tuple in tqdm(loaded_set):
    code, id = club_info_tuple
    if id not in league_dict:
        season_dict = {}
        league_page_url = 'https://www.transfermarkt.com/' + code + '/' + 'platzierungen' + '/' + 'verein' + '/' + id
        club_soup = create_soup(league_page_url)
        tables = club_soup.find_all('tbody')
        if len(tables) > 1:
            table = tables[1]
            table_entries = table.find_all('tr')

            for entry in table_entries[:6]:
                season = entry.find('td', class_='zentriert').string
                league = entry.find('td', class_='no-border-links hauptlink').find('a').get('href').split('/')[4]
                season_dict[season] = league
            league_dict[code] = season_dict

100%|██████████| 657/657 [27:36<00:00,  2.52s/it]  


In [8]:
# saving the set to a pickle file
# sys.setrecursionlimit(10**6)
# with open('league_dict_v2.pkl', 'wb') as file:
#     pickle.dump(league_dict, file)

# loading the set from the pickle file
# with open('league_dict_v2.pkl', 'rb') as file:
#     league_dict = pickle.load(file)

# converting the dictionary to a df
# league_df = pd.DataFrame(league_dict)

# the pkl file has now been deleted 
# converting the league df to a parquet file and loading it back in as a dictionary
# league_df.to_parquet('league_dict_v2.parquet')

# loading parquet file back into a df and then converting to a dictionary
df_loaded = pd.read_parquet('data/league_dict.parquet')
league_dict = df_loaded.to_dict()

### Scraping the transfers based on the league dictionary

In [ ]:
years = ["2018", "2019", "2020", "2021", "2022"]
seasons = ["18/19","19/20", "20/21", "21/22", "22/23"]
dfs = []
league_id = 'FR1'
league_code = 'ligue-1'

for year, main_season in zip(years, seasons):

    url = f'https://www.transfermarkt.com/{league_code}/transfers/wettbewerb/{league_id}/plus/?saison_id={year}&s_w=&leihe=3&intern=0&intern=1'
    soup = create_soup(url)

    players = []
    selling_clubs = []
    buying_clubs = []
    selling_leagues = []
    buying_leagues = []
    fees = []

    boxes = soup.find('div', class_='large-8 columns').find_all('div', class_='box')

    for box in tqdm(boxes[3:]):
        club_name = box.find('h2', class_='content-box-headline content-box-headline--inverted content-box-headline--logo').find_all('a')[1].get('href').split('/')[1]
        print(f'club_name: {club_name}')
        tables = box.find_all('div', class_='responsive-table')
        ins = tables[0].find('tbody').find_all('tr')
        outs = tables[1].find('tbody').find_all('tr')

        if len(ins) > 1:
            for row in ins:
                fee = row.find_all('td', class_='rechts')[1].string
                if isinstance(fee, str):
                    fee_first = fee[0]
                    fee_last = fee[-1]
                    if fee_first == "€":
                        fee_number = fee[1:-1]
                        if fee_last == "m":
                            fee_value = float(fee_number)*10**6
                        elif fee_last == 'k':
                            fee_value = float(fee_number)*10**3

                        player_name = row.find('span', class_='hide-for-small').string
                        club_section = row.find('td', class_='no-border-links verein-flagge-transfer-cell').find('a')
                        from_club_code = club_section.get('href').split('/')[1]
                        # print(from_club)
                        if from_club_code in league_dict:
                            if main_season in league_dict[from_club_code]:
                                selling_league = league_dict[from_club_code][main_season]
                            else:
                                selling_league = '-'
                        else:
                            selling_league = '-'
                        
                        players.append(player_name)
                        selling_clubs.append(from_club_code)
                        buying_clubs.append(club_name)
                        selling_leagues.append(selling_league)
                        buying_leagues.append(league_id)
                        fees.append(fee_value)
                    
        if len(outs) > 1:
            for row in outs:
                fee = row.find_all('td', class_='rechts')[1].string
                if isinstance(fee, str):
                    fee_first = fee[0]
                    fee_last = fee[-1]
                    if fee_first == "€":
                        fee_number = fee[1:-1]
                        if fee_last == "m":
                            fee_value = float(fee_number)*10**6
                        elif fee_last == 'k':
                            fee_value = float(fee_number)*10**3
                    
                        player_name = row.find('span', class_='hide-for-small').string
                        club_section = row.find('td', class_='no-border-links verein-flagge-transfer-cell').find('a')
                        to_club_code = club_section.get('href').split('/')[1]
                        # print(to_club)
                        if to_club_code in league_dict:
                            if main_season in league_dict[to_club_code]:
                                buying_league = league_dict[to_club_code][main_season]
                            else:
                                buying_league = '-'
                        else:
                            buying_league = '-'

                        players.append(player_name)
                        selling_clubs.append(club_name)
                        buying_clubs.append(to_club_code)
                        selling_leagues.append(league_id)
                        buying_leagues.append(buying_league)
                        fees.append(fee_value)
    
    # Define a larger DataFrame
    data = {'Season': [main_season]*len(players),
            'Player': players, 
            'From_Club': selling_clubs,
            'To_Club': buying_clubs,
            'From_League': selling_leagues,
            'To_League': buying_leagues,
            'Fee': fees}
    df = pd.DataFrame(data)
    dfs.append(df)

In [10]:
main_df = pd.concat(dfs)

# Display the DataFrame using df.head()
main_df.iloc[0:50]

,Season,Player,From_Club,To_Club,From_League,To_League,Fee
0,18/19,Saman Ghoddos,ostersunds-fk,amiens-sc,SE1,FR1,4000000.0
1,18/19,Juan Otero,club-estudiantes-de-la-plata,amiens-sc,AR1N,FR1,2140000.0
2,18/19,Rafal Kurzawa,gornik-zabrze,amiens-sc,PL1,FR1,850000.0
3,18/19,Tanguy Ndombélé,amiens-sc,olympique-lyon,FR1,FR1,8000000.0
4,18/19,Guessouma Fofana,amiens-sc,ea-guingamp,FR1,FR1,1000000.0
5,18/19,Harrison Manzala,amiens-sc,sco-angers,FR1,FR1,1000000.0
6,18/19,Jeff Reine-Adélaïde,fc-arsenal-u23,sco-angers,None,FR1,1600000.0
7,18/19,Harrison Manzala,amiens-sc,sco-angers,FR1,FR1,1000000.0
8,18/19,Vincent Pajot,as-saint-etienne,sco-angers,FR1,FR1,1000000.0
9,18/19,Dorian Bertrand,so-cholet,sco-angers,FR3,FR1,800000.0


In [35]:
# saving dataframe as csv file
main_df.to_csv('L1_transfers_v2.csv', index=False)

In [36]:
# combining csv files
file_names = ['PL_transfers_v2.csv', 'LL_transfers_v2.csv', 'BL_transfers_v2.csv', 'SA_transfers_v2.csv', 'L1_transfers_v2.csv']
dfs = []

for file_name in file_names:
    df = pd.read_csv(file_name)
    dfs.append(df)

combined_df = pd.concat(dfs, ignore_index=True)
combined_df.to_csv('all_transfers_v2.csv', index=False)

In [37]:
# removing duplicate rows from the final data
df = pd.read_csv('all_transfers_v2.csv')
df.drop_duplicates(inplace=True)
df.to_csv('all_transfers_no_duplicates.csv', index=False)

In [ ]:
# dictionary for looking up league IDs
league_id_dict = {'L1': 'Bundesliga', 'GB1': 'Premier League', 'FR1': 'Ligue 1', 'IT1': 'Serie A', 'ES1': 'La Liga'}

### Adding missing leagues

In [3]:
df = pd.read_csv('data/all_transfers_no_duplicates.csv')
df.head()

,Season,Player,From_Club,To_Club,From_League,To_League,Fee
0,18/19,Riyad Mahrez,leicester-city,manchester-city,GB1,GB1,67800000.0
1,18/19,Ante Palaversa,hnk-hajduk-split,manchester-city,KR1,GB1,6300000.0
2,18/19,Philippe Sandler,pec-zwolle,manchester-city,NL1,GB1,2500000.0
3,18/19,Ko Itakura,kawasaki-frontale,manchester-city,JAP1,GB1,1100000.0
4,18/19,Daniel Arzani,melbourne-city-fc,manchester-city,AUS1,GB1,890000.0


In [395]:
df[df['From_League'].isna()]

,Season,Player,From_Club,To_Club,From_League,To_League,Fee


In [393]:
df[df['To_Club'] == 'wuhan-fc']

,Season,Player,From_Club,To_Club,From_League,To_League,Fee
793,18/19,Léo Baptistão,espanyol-barcelona,wuhan-fc,ES1,CSL,5800000.0
926,19/20,Daniel Carriço,fc-sevilla,wuhan-fc,ES1,CSL,2000000.0


In [396]:
set(df[df['To_League'].isna()]['To_Club'])

set()

In [250]:
mask = (df['Season'] == '20/21') & (df['From_Club'] == 'real-madrid-b-castilla-')
df.loc[mask, 'From_League'] = 'ES3A'

In [392]:
mask = (df['To_Club'] == 'wuhan-fc') 
df.loc[mask, 'To_League'] = 'CSL'

In [397]:
df.to_csv('all_transfers_no_duplicates_updated.csv', index=False)

In [ ]:
df.to_parquet('all_transfers_no_duplicates_updated.parquet')

### Scraping aggregate data for bar charts

In [27]:
years = ["2018", "2019", "2020", "2021", "2022"]
seasons = ["18/19","19/20", "20/21", "21/22", "22/23"]
dfs = []
league_id = 'FR1'
league_code = 'ligue-1'

for year, main_season in zip(years, seasons):

    url = f'https://www.transfermarkt.com/{league_code}/transfers/wettbewerb/{league_id}/plus/?saison_id={year}&s_w=&leihe=3&intern=0&intern=1'
    soup = create_soup(url)

    club_codes = []
    club_names = []
    expenditures = []
    incomes = []

    boxes = soup.find('div', class_='large-8 columns').find_all('div', class_='box')
    for box in tqdm(boxes[3:]):
        club_code = box.find('h2', class_='content-box-headline content-box-headline--inverted content-box-headline--logo').find_all('a')[1].get('href').split('/')[1]
        club_name = box.find('h2', class_='content-box-headline content-box-headline--inverted content-box-headline--logo').find('a').get('title')
        if main_season == '20/21' and club_name == 'Real Madrid':
            exp_val = '0'
        else:
            exp_string = box.find('span', class_='transfer-einnahmen-ausgaben redtext').string
            exp_val = exp_string.split(':')[1].strip()
        # print(exp_val)
        if len(exp_val) > 1:
            exp = float(exp_val[1:-1]) * 10**6
        else:
            exp = float(exp_val)

        inc_string = box.find('span', class_='transfer-einnahmen-ausgaben greentext').string
        inc_val = inc_string.split(':')[1].strip()
        # print(inc_val)
        if len(inc_val) > 1:
            inc = float(inc_val[1:-1]) * 10**6
        else: inc = float(inc_val)

        club_codes.append(club_code)
        club_names.append(club_name)
        expenditures.append(exp)
        incomes.append(inc)
            
    data = {'Season': [main_season]*len(club_codes),
            'League': [league_id]*len(club_codes),
            'club_code': club_codes, 
            'club_name': club_names,
            'Expenditure': expenditures,
            'Income': incomes}
    df = pd.DataFrame(data)
    dfs.append(df)

100%|██████████| 20/20 [00:00<00:00, 392.60it/s]


In [30]:
main_df = pd.concat(dfs)

# Display the DataFrame using df.head()
main_df.iloc[50:100]

,Season,League,club_code,club_name,Expenditure,Income
10,20/21,FR1,sco-angers,Angers SCO,11250000.0,10700000.0
11,20/21,FR1,fc-girondins-bordeaux,FC Girondins Bordeaux,0.0,17000000.0
12,20/21,FR1,fc-nantes,FC Nantes,5000000.0,2000000.0
13,20/21,FR1,stade-brest-29,Stade Brestois 29,16100000.0,12000000.0
14,20/21,FR1,fc-metz,FC Metz,14700000.0,10000000.0
15,20/21,FR1,dijon-fco,Dijon FCO,15800000.0,17000000.0
16,20/21,FR1,as-saint-etienne,AS Saint-Étienne,6200000.0,44100000.0
17,20/21,FR1,nimes-olympique,Nîmes Olympique,11250000.0,2500000.0
18,20/21,FR1,fc-lorient,FC Lorient,26500000.0,6500000.0
19,20/21,FR1,rc-lens,RC Lens,19000000.0,0.0


In [31]:
# saving dataframe as csv file
main_df.to_csv('L1_exp_inc.csv', index=False)

In [33]:
# combining csv files
file_names = ['PL_exp_inc.csv', 'LL_exp_inc.csv', 'BL_exp_inc.csv', 'SA_exp_inc.csv', 'L1_exp_inc.csv']
dfs = []

for file_name in file_names:
    df = pd.read_csv(file_name)
    dfs.append(df)

combined_df = pd.concat(dfs, ignore_index=True)
combined_df.to_csv('exp_inc.csv', index=False)